In [1]:
import ipywidgets as ipw
import subprocess

In [2]:
from utils.control import *

In [ ]:
msg,updates_needed,config = check_for_updates('s1267')
#tutto = check_for_updates()

In [ ]:
 msg

In [ ]:
config

In [ ]:
updates_needed

In [ ]:
cscs_username="cpi"
remotehost='daint.alps.cscs.ch'
alps_files=""
conda_init = f"source /users/{cscs_username}/miniconda3/bin/activate"
commands = [
    ["ssh", remotehost, "if [ ! -f Miniconda3-latest-Linux-aarch64.sh ]; then wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-aarch64.sh; fi"],
    ["ssh", remotehost, f"if [ ! -d /users/{cscs_username}/miniconda3 ]; then bash Miniconda3-latest-Linux-aarch64.sh -b -p /users/{cscs_username}/miniconda3; fi"],
    ["scp", f"{alps_files}bashrc_template", f"{remotehost}:/users/{cscs_username}"],
    ["ssh", remotehost, "mv .bashrc .old_bashrc"],
    ["ssh", remotehost, "mv bashrc_template .bashrc"],
    ["ssh", remotehost, f"bash -l -c '{conda_init} && conda env list | grep -q \"^phonopy \" && echo \"Environment exists\" || conda create -n phonopy -c conda-forge phonopy seekpath'"],
]
output, success = run_command(commands[-1])

if success:
    print("✅ Command executed successfully")
else:
    print(f"❌ Error: {output}")


In [7]:
def load_yaml_commands(file_path="/home/jovyan/opt/aiidalab-alps-files/config.yml"):
    """Load custom commands from a YAML file."""
    with open(file_path, "r") as file:
        return yaml.safe_load(file)
def execute_custom_commands(cscs_username, remotehost, qe_uenv="", python_version=""):
    """Execute all commands from custom_commands in the YAML file."""
    yaml_commands = load_yaml_commands()
    
    if "custom_commands" not in yaml_commands:
        print("❌ No custom commands found in YAML file. Exiting.")
        return False
    
    variables = yaml_commands["custom_commands"].get("variables", {})
    conda_init = variables.get("conda_init", "source /users/{cscs_username}/miniconda3/bin/activate").format(cscs_username=cscs_username)
    config_files = variables.get("config_files")
    # Execute environment setup commands
    env_setup_commands = yaml_commands["custom_commands"].get("env_setup_commands", {})
    for setup_name, commands in env_setup_commands.items():
        for entry in commands:
            formatted_command = entry["command"].format(
                cscs_username=cscs_username,
                remotehost=remotehost,
                qe_uenv=qe_uenv,
                python_version=python_version,
                conda_init=conda_init,
                config_files=config_files
            )
            ssh_command = ["ssh", remotehost, formatted_command] if entry["type"] == "ssh" else formatted_command.split()
            output, success = run_command(ssh_command)
            if not success:
                print(f"❌ Failed to execute: {formatted_command}. Exiting, ask for help.")
                return False
    
    # Execute special setup commands
    special_commands = yaml_commands["custom_commands"].get("special_commands", {})
    for setup_name, commands in special_commands.items():
        for entry in commands:
            if isinstance(entry, dict):
                command_type = entry.get("type", "shell")
                command = entry.get("command", "")
            else:
                command_type = "shell"
                command = entry

            formatted_command = command.format(
                cscs_username=cscs_username,
                remotehost=remotehost,
                qe_uenv=qe_uenv,
                python_version=python_version,
                conda_init=conda_init,
                config_files=config_files
            )
            
            ssh_command = ["ssh", remotehost, formatted_command] if command_type == "ssh" else formatted_command.split()
            output, success = run_command(ssh_command)
            if not success:
                print(f"❌ Failed to execute: {formatted_command}. Exiting, ask for help.")
                return False
    
    return True

In [8]:
execute_custom_commands("cpi", "daint.alps.cscs.ch", qe_uenv="quantumespresso/v7.4:v2", python_version="3.9.13")


if [ ! -f Miniconda3-latest-Linux-aarch64.sh ]; then wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-aarch64.sh; fi


if [ ! -d /users/{cscs_username}/miniconda3 ]; then bash Miniconda3-latest-Linux-aarch64.sh -b -p /users/{cscs_username}/miniconda3; fi


scp {config_files}/bashrc_template {remotehost}:/users/{cscs_username}


mv .bashrc .old_bashrc


mv bashrc_template .bashrc


{conda_init} && conda env list | grep -q "^phonopy " && echo "Environment exists" || conda create -n phonopy -c conda-forge phonopy seekpath


if [ ! -d 'critic2' ]; then git clone https://github.com/aoterodelaroza/critic2.git; fi


if [ ! -f /users/{cscs_username}/critic2/build/src/critic2 ]; then cd critic2 && mkdir -p build && cd build && uenv run {qe_uenv} cmake .. && uenv run {qe_uenv} make; fi


ls /users/{cscs_username}/critic2/build/src/critic2



True